In [1]:
#@title Copyright 2020 The Earth Engine Community Authors { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# An Intro to the Earth Engine Python API

Author: guiattard

Within the last decade, a large amount of geospatial data, such as satellite data (e.g. land surface temperature, vegetation) or the output of large scale, even global models (e.g. wind speed, groundwater recharge), have become freely available from multiple national agencies and universities (e.g. NASA, USGS, NOAA, and ESA). These geospatial data are used every day by scientists and engineers of all fields, to predict weather, prevent disasters, secure water supply, or study the consequences of climate change. When using these geospatial data, a few questions arise:

- What data are available and where can it be found?
- How can we access these data?
- How can we manipulate these petabytes of data?

In this tutorial, an introduction to the [Google Earth Engine Python API](https://developers.google.com/earth-engine/guides/python_install) is presented. After some setup and some exploration of the Earth Engine Data Catalog, we’ll see how to handle geospatial datasets with [pandas](https://pandas.pydata.org/) and make some plots with matplotlib.

First, we’ll see how to get the timeseries of a variable for a region of interest. An application of this procedure will be done to extract land surface temperature in an urban and a rural area near the city of Lyon, France to illustrate the heat island effect. Secondly, we will detail procedures for static mapping and exporting results as a GeoTIFF.

Finally, the folium library will be introduced to make interactive maps. In this last part, we’ll see how to include some GEE datasets as tile layers of a folium map.

## Exploration of the Earth Engine Data Catalog

Have you ever thought that getting a meteorological dataset could be as easy as finding the nearest pizzeria? To convince you, visit the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog) and explore datasets using the search bar or browsing by tag.

Let's say that we need to know the elevation of a region, some soil properties (e.g. clay, sand, silt content) and some meteorological observations (e.g. temperature, precipitation, evapotranspiration). Well, inside the Earth Engine Catalog we find:

- [SRTM global elevation](https://developers.google.com/earth-engine/datasets/catalog/USGS_SRTMGL1_003) with a resolution of 30 m,
- [OpenLandMap datasets](https://developers.google.com/earth-engine/datasets/catalog/OpenLandMap_SOL_SOL_CLAY-WFRACTION_USDA-3A1A1A_M_v02) with soil properties at a resolution of 250 m (e.g. clay, sand, and silt content), and
- [GRIDMET](https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_GRIDMET) temperature, precipitation, and evapotranspiration, for example.

Of course the resolution, frequency, spatial and temporal extent, as well as data source (e.g. satellite image, interpolated station data, or model output) vary from one dataset to another. Therefore, read the description carefully and make sure you know what kind of dataset you are selecting!


## Run me first

First of all, run the following cell to initialize the API. The output will contain instructions on how to grant this notebook access to Earth Engine using your account.

In [2]:
import ee
from google.colab import auth
# Trigger the authentication flow.
ee.Authenticate()
auth.authenticate_user(project_id="arizona-whirl")
# Initialize the library.
ee.Initialize(project='arizona-whirl')

In [3]:
print(ee.Number(1).add(1).getInfo())

2


In [4]:
# Import the nighttimelight collection
viirs = ee.ImageCollection("NASA/VIIRS/002/VNP46A2").select("Gap_Filled_DNB_BRDF_Corrected_NTL")


In [15]:
import pandas as pd
cities_fc = ee.FeatureCollection("projects/arizona-whirl/assets/fl_cities")
cities = {
    'Miami': cities_fc.filter(ee.Filter.eq('NAME', 'Miami')).geometry(),
    'Tampa': cities_fc.filter(ee.Filter.eq('NAME', 'Tampa')).geometry(),
    'Orlando': cities_fc.filter(ee.Filter.eq('NAME', 'Orlando')).geometry(),
    'Jacksonville': cities_fc.filter(ee.Filter.eq('NAME', 'Jacksonville')).geometry(),
    'Fort Myers': cities_fc.filter(ee.Filter.eq('NAME', 'Fort Myers')).geometry()
}

hurricane_dates = pd.DataFrame({
    'name': ['Hermine', 'Matthew', 'Irma', 'Michael', 'Sally','Ian', 'Idalia', 'Debby', 'Helene', 'Milton'],
    'date': ['2016-09-02', '2016-10-7', '2017-09-10', '2018-10-10', '2020-09-16', '2022-09-28', '2023-08-30', '2024-08-05', '2024-09-27', '2024-10-10']
})


In [18]:
def get_baseline(city_geom, hurricane_date):
    start = ee.Date(hurricane_date).advance(-30, 'day')
    end = ee.Date(hurricane_date)

    baseline_img = viirs.filterDate(start, end).mean().select("Gap_Filled_DNB_BRDF_Corrected_NTL")

    baseline_val = baseline_img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=city_geom,
        scale=500
    ).get("Gap_Filled_DNB_BRDF_Corrected_NTL")

    return baseline_val

def daily_pct_recovery(city_geom, hurricane_date, baseline_val):
    start = ee.Date(hurricane_date)
    end = start.advance(30, 'day')
    images = viirs.filterDate(start, end).select("Gap_Filled_DNB_BRDF_Corrected_NTL")

    def calc_pct(image):
        mean_val = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=city_geom,
            scale=500
        ).get("Gap_Filled_DNB_BRDF_Corrected_NTL")

        # Use ee.Algorithms.If to check for nulls directly
        pct_recovered = ee.Algorithms.If(
            ee.Algorithms.IsEqual(baseline_val, None),
            None,  # skip if baseline is null
            ee.Algorithms.If(
                ee.Algorithms.IsEqual(mean_val, None),
                None,  # skip if mean_val is null
                ee.Number(mean_val).divide(baseline_val).multiply(100)  # otherwise calculate %
            )
        )

        return image.set('pct_recovered', pct_recovered)

    return images.map(calc_pct)


In [21]:
from google.colab import files
all_data = []

for city_name, city_geom in cities.items():
    for idx, row in hurricane_dates.iterrows():
        baseline_val = get_baseline(city_geom, row['date'])
        daily_images = daily_pct_recovery(city_geom, row['date'], baseline_val)

        # Convert to list for export
        pct_list = daily_images.aggregate_array('pct_recovered').getInfo()
        for day, pct in enumerate(pct_list):
            all_data.append({
                'city': city_name,
                'hurricane': row['name'],
                'date': row['date'],
                'day_after': day,
                'percent_recovered': pct
            })

df = pd.DataFrame(all_data)
df.to_csv('hurricane_recovery.csv', index=False)
files.download("hurricane_recovery.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Documentation

- The full documentation of the Google Earth Engine Python API is available [here](https://developers.google.com/earth-engine/api_docs).
- The Google Earth Engine User Guide is available [here](https://developers.google.com/earth-engine).
- Some tutorials are available [here](https://developers.google.com/earth-engine/tutorials).
- An example based on the Google Earth Engine Javascript console dedicated to Land Surface Temperature estimation is provided in the open access supplementary material of [Benz et al., (2017)](https://iopscience.iop.org/article/10.1088/1748-9326/aa5fb0/meta). You can access the code [here](https://code.earthengine.google.com/4a1bc64dbc3351a1e364490758d4cf2d).

## Acknowledgements

Thanks to Susanne Benz and Justin Braaten for reviewing and helping write this tutorial.